In [1]:
! which python

/Users/no_lineal/opt/anaconda3/envs/research36/bin/python


In [2]:
import pandas as pd
import numpy as np
import math

from scipy import signal
from scipy.integrate import simps

from mne_features import univariate

import mat73

from tqdm import tqdm

import os

import warnings
warnings.filterwarnings('ignore')

In [3]:
"""

    where am i?

"""

PATH = os.getcwd() + '/'
data_path = PATH + 'data/'

print(f'PATH: {PATH}')
print(f'data path: {data_path}')

PATH: /Users/no_lineal/Documents/GitHub/eeg_processing/
data path: /Users/no_lineal/Documents/GitHub/eeg_processing/data/


In [4]:
"""

    load data

"""

# signal
folder_name = 'LFP_conditions/'

# dorsal
d_cong = mat73.loadmat( data_path + folder_name + 'dcong.mat' )['dcong']
d_incg = mat73.loadmat( data_path + folder_name + 'dincg.mat' )['dincg']

In [5]:
"""

    dorsal trials

"""

d_cong_trial = sorted( [ x for x in d_cong.keys() if 'STN' in x ] )
d_incg_trial = sorted( [ x for x in d_incg.keys() if 'STN' in x ] )

trials = sorted(list(set(d_cong_trial).intersection( set(d_incg_trial) )))

print( f'd_cong_trial: {d_cong_trial}' )
print( f'd_incong_trial: {d_incg_trial}' )
print( f'trials: {trials}' )

d_cong_trial: ['STN02', 'STN03', 'STN07a', 'STN08', 'STN10a', 'STN10b', 'STN11', 'STN13']
d_incong_trial: ['STN02', 'STN07a', 'STN08', 'STN10a', 'STN10b', 'STN11', 'STN13']
trials: ['STN02', 'STN07a', 'STN08', 'STN10a', 'STN10b', 'STN11', 'STN13']


In [6]:
"""

    signal features

"""

def comp_signal( signals ):
    
    columns_names = [
        'std',
        'mean', 
        'skewness', 
        'h_mobility',
        'h_complexity'
    ]
    
    feat = []
    
    for s in range(signals.shape[0]):
        
        tmp = signals[s]
        tmp = tmp[ ~np.isnan(tmp) ]
        
        len_ = len(tmp)
        
        std = univariate.compute_std( tmp )
        
        if not std:
            std = 0.0
        
        mean = univariate.compute_mean( tmp )
        
        skw = univariate.compute_skewness( tmp )
        
        h_mob = univariate.compute_hjorth_mobility( tmp )
        
        if math.isnan( h_mob ):
            h_mob = 0.0
        
        h_comp = univariate.compute_hjorth_complexity( tmp )
        
        if math.isnan( h_comp ):
            h_comp = 0.0
        
        feat.append( [std, mean, skw, h_mob, h_comp] )
        
    return pd.DataFrame(feat, columns = columns_names)

In [7]:
"""

    bandpower

"""

def band_power(signals, band, sf):
    
    columns_names = [
        'len',
        'band_power'
    ]
    
    feat = []
    
    for s in range(signals.shape[0]):
        
        # drop nan
        tmp = signals[s]
        tmp = tmp[ ~np.isnan(tmp) ]
        tmp = tmp[-300:]
        
        # length
        len_ = len(tmp)
        
        # window
        nperseg = (2 / band[0]) * sf
        
        # welch's periodogram
        freqs, psd = signal.welch(tmp, sf, nperseg=nperseg)
        
        # ub/lb
        low, high = band[0], band[1]
        
        # Frequency resolution
        freq_res = freqs[1] - freqs[0]
        
        # Find closest indices of band in frequency vector
        idx_band = np.logical_and(freqs >= low, freqs <= high)
        
        # simpson's integral approximation
        bp = simps( psd[idx_band], dx=freq_res )
        
        feat.append( [len_, bp] )
        
    return pd.DataFrame(feat, columns = columns_names)

In [8]:
"""

    adjust signal

"""

# response
cue_type = 'sigcue'

# sampling frequency
sf = 1000

cong_bp = pd.DataFrame()
incg_bp = pd.DataFrame()

for s in tqdm(trials):
    
    #print(f'\nsmaple subject: {s}')
    
    cue_cong = d_cong[ s ][ cue_type ][0][ 'dCong_trl' ]
    cue_incg = d_incg[ s ][ cue_type ][0][ 'dIncg_trl' ]
    
    #print(f'cong: {cue_cong.shape}')
    #print(f'incg: {cue_incg.shape}')
    
    # congruent
    delta_cong = band_power( cue_cong, [0.5, 4.0], sf ).rename(columns={'band_power': 'delta_power'})
    theta_cong = band_power( cue_cong, [4.0, 8.0], sf ).rename(columns={'band_power': 'theta_power'})
    alpha_cong = band_power( cue_cong, [8.0, 12.0], sf ).rename(columns={'band_power': 'alpha_power'})
    beta_cong = band_power( cue_cong, [12.0, 30.0], sf ).rename(columns={'band_power': 'beta_power'})
    gamma_cong = band_power( cue_cong, [30.0, 100.0], sf ).rename(columns={'band_power': 'gamma_power'})
    
    power_band_cong = pd.concat(
        [
            delta_cong[['delta_power']],
            theta_cong[['theta_power']],
            alpha_cong[['alpha_power']],
            beta_cong[['beta_power']], 
            gamma_cong[['gamma_power']]
        ], axis=1
    )
    
    # feature signals
    feat_cong = comp_signal( cue_cong )
    
    power_band_cong = pd.concat([ power_band_cong, feat_cong ], axis=1)
    
    power_band_cong['subject'] = s
    
    if cong_bp.empty:
        
        cong_bp = power_band_cong
        
    else:
        
        cong_bp = pd.concat( [ cong_bp, power_band_cong ] )
    
    #print(power_band_cong.info())
    #print(power_band_cong.head())
    
    # incongruent
    delta_incg = band_power( cue_incg, [0.5, 4.0], sf ).rename(columns={'band_power': 'delta_power'})
    theta_incg = band_power( cue_incg, [4.0, 8.0], sf ).rename(columns={'band_power': 'theta_power'})
    alpha_incg = band_power( cue_incg, [8.0, 12.0], sf ).rename(columns={'band_power': 'alpha_power'})
    beta_incg = band_power( cue_incg, [12.0, 30.0], sf ).rename(columns={'band_power': 'beta_power'})
    gamma_incg = band_power( cue_incg, [30.0, 100.0], sf ).rename(columns={'band_power': 'gamma_power'})
    
    power_band_incg = pd.concat(
        [
            delta_incg[['delta_power']],
            theta_incg[['theta_power']],
            alpha_incg[['alpha_power']],
            beta_incg[['beta_power']], 
            gamma_incg[['gamma_power']]
        ], axis=1
    )
    
    # feature signals
    feat_incg = comp_signal( cue_incg )
    
    power_band_incg = pd.concat([ power_band_incg, feat_incg ], axis=1)
    
    power_band_incg['subject'] = s
    
    if incg_bp.empty:
        
        incg_bp = power_band_incg
        
    else:
        
        incg_bp = pd.concat([ incg_bp, power_band_incg ])
    
    #print(power_band_incg.info())
    #print(power_band_incg.head())
    
print(f'\ncongruent: ')
print(cong_bp.info())
print(cong_bp.head())

print(f'\nincongruent: ')
print(incg_bp.info())
print(incg_bp.head())

100%|██████████| 7/7 [00:01<00:00,  6.66it/s]


congruent: 
<class 'pandas.core.frame.DataFrame'>
Int64Index: 638 entries, 0 to 83
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   delta_power   638 non-null    float64
 1   theta_power   638 non-null    float64
 2   alpha_power   638 non-null    float64
 3   beta_power    638 non-null    float64
 4   gamma_power   638 non-null    float64
 5   std           638 non-null    float64
 6   mean          638 non-null    float64
 7   skewness      638 non-null    float64
 8   h_mobility    638 non-null    float64
 9   h_complexity  638 non-null    float64
 10  subject       638 non-null    object 
dtypes: float64(10), object(1)
memory usage: 59.8+ KB
None
   delta_power  theta_power  alpha_power   beta_power  gamma_power  \
0          0.0          0.0  4758.662269  2640.061771   364.112271   
1          0.0          0.0   442.979556   341.863066   155.905377   
2          0.0          0.0   712.150198   232.794823  

In [9]:
"""

    save dataframe

"""

cong_bp['label'] = 0
incg_bp['label'] = 1

data = pd.concat([cong_bp, incg_bp])

print(data.info())
print(data.head())

data.to_csv( data_path + 'lfp_features.csv', index=False )

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1241 entries, 0 to 75
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   delta_power   1241 non-null   float64
 1   theta_power   1241 non-null   float64
 2   alpha_power   1241 non-null   float64
 3   beta_power    1241 non-null   float64
 4   gamma_power   1241 non-null   float64
 5   std           1241 non-null   float64
 6   mean          1241 non-null   float64
 7   skewness      1241 non-null   float64
 8   h_mobility    1241 non-null   float64
 9   h_complexity  1241 non-null   float64
 10  subject       1241 non-null   object 
 11  label         1241 non-null   int64  
dtypes: float64(10), int64(1), object(1)
memory usage: 126.0+ KB
None
   delta_power  theta_power  alpha_power   beta_power  gamma_power  \
0          0.0          0.0  4758.662269  2640.061771   364.112271   
1          0.0          0.0   442.979556   341.863066   155.905377   
2          0.